# GP Cartpole Server

This notebook introduces the genetic programming (GP) approach for the cartpole REST interface. GP creates a program for the interface's controller to reply to a client's requests on the cartpole REST interface.

The figure illustrates the position of the GP software within the REST interface implementation.  

![REST gp design](http://www.plantuml.com/plantuml/png/5SWx3i8m303Ggy05uW4cPjYO2dlLEbOQ9R4ZVyJrKT_uhfMnHwRGVZdSfpq0TFkbKls8FOmKXPtO4ye0p8Jjfl1StlVbzVd2sCn9WSLf5sa6vwvbhfci5aun-Xy0)

The [OpenAPI generator](https://github.com/OpenAPITools/openapi-generator) created the REST interface and the REST controller prototype from the [cartpole API spec](https://raw.githubusercontent.com/cdeck3r/DRL4REST/master/src/cartpole/spec/OpenAPIv1.json). In this approach we replace the controller's empty function body with an implementation found through genetic programming. Additionally, we introduce a ServerModel, which keeps a state if required. One may access the state using CRUD operations. 

## Example: `cart_get()` Controller Function

As a example, look at the `cart_get()` controller prototype the OpenAPI generator created for the cartpole REST interface.  

```python
def cart_get():  # noqa: E501
    """cart_get

     # noqa: E501


    :rtype: Cart
    """
    return 'do some magic!'
```

A concrete example implementation would look like the following:

```python
def cart_get():  # noqa: E501
    # just return a Cart object with random values

    position = random()
    velocity = random()
    direction = choice(["left", "right"]) 

    return Cart(position=position, velocity=velocity, direction=direction)
```

The `Cart` class is one the generated models. The contructor defines the params as follows:
```python
def __init__(self, position=None, velocity=None, direction=None):  # noqa: E501
    """Cart - a model defined in OpenAPI

    :param position: The position of this Cart.  # noqa: E501
    :type position: float
    :param velocity: The velocity of this Cart.  # noqa: E501
    :type velocity: float
    :param direction: The direction of this Cart.  # noqa: E501
    :type direction: str
    """
    ...
```

## Genetic Program Design

For the `get_cart()` example from above, the implementation idea is, if there already is a cart object, then it will return this. Otherwise, it creates a new one and returns it. Here's the pseudo code:

```python
if cart exists 
    return cart
else
    # params
    pos = get_init_cart_position
    vel = get_init_cart_velocity
    dir = get_init_cart_direction
    # return the newly created cart 
    return new Cart with params above
```

We test run the code within the `evaluate()` function of the GP algorithm. It needs 

1. to run without any exception and 
1. repeated `get_cart()` requests shall result in the same response.

## Genetic Program Implementation

The GP bases on the Artificial Ant Problem from the [DEAP example site](https://deap.readthedocs.io/en/master/examples/gp_ant.html).

### Import libs

DEAP was previously installed in this image. If not, run `pip install deap`.

In [1]:
import random

import numpy

from functools import partial

from deap import algorithms
from deap import base
from deap import creator
from deap import tools
from deap import gp

The OpenAPI generated interface code is based on python-flask. If necessary, activate the following cell `ESC + y` and install the requirements.

In [2]:
# import the models generated by OpenAPI
import sys
sys.path.append("/DRL4REST/openapi/cartpole/python-flask")

from openapi_server.models import Cart

In [3]:
c=Cart()
type(c)
c.position=10
c.position
c.to_dict()

{'position': 10, 'velocity': None, 'direction': None}

### Functions

These are generic functions, e.g. for control flow and concatenating functions with each other, and the specific function for the server code. 

#### Generic Functions

In [4]:
def progn(*args):
    for arg in args:
        arg()

def prog2(out1, out2): 
    return partial(progn,out1,out2)

def prog3(out1, out2, out3):     
    return partial(progn,out1,out2,out3)

def if_then_else(condition, out1, out2):
    out1() if condition() else out2()

### Controller Functions

These are the functions the GP algorithm strings together utilizing the generic functions from above. The created program forms an individual and is tested by the `evaluate()` function.   

In [5]:
# ...

#### ServerModel CRUD Functions

`CartpoleServer` is the implementation of the ServerModel. The ServerModel is a static class. Controller utilize the model to store state between successive calls. State access and manipulation via CRUD.

In [6]:
from openapi_server.models import Cart

class CartpoleServer(object):

    # state as class variable
    _cart = None

    # set the class in a defined (initial) state
    @classmethod
    def reset(cls):
        cls._cart = None
    
    @classmethod
    def create_cart(cls):
        if isinstance(cls._cart, type(None)):
            position = random.random()
            velocity = random.random()
            # we know the values from the models, e.g.
            #   models/cart.py or models/direction.py
            direction = random.choice(["left", "right"])
            cls._cart = Cart(position=position, velocity=velocity, direction=direction)
        
        return cls._cart

    @classmethod
    def read_cart(cls):
        return cls._cart
    
    @classmethod
    def update_cart(cls):
        pass

    @classmethod
    def delete_cart(cls):
        cls._cart = None

In [7]:
# small test routine for the CartpoleServer

def test_CartpoleServer():
    s=CartpoleServer
    s.reset()

    c=s.read_cart()
    assert type(c) == type(None)
    
    s.create_cart()
    c=s.read_cart()
    assert isinstance(type(c), type(Cart))
    
    c2=s.read_cart()
    assert c == c2
    assert c.to_dict() == c2.to_dict()

    s.delete_cart()
    c=s.read_cart()
    assert type(c) == type(None)
    
    s.create_cart()
    c=s.read_cart()
    assert isinstance(type(c), type(Cart))
    c2=s.read_cart()
    assert c == c2
    assert c.to_dict() == c2.to_dict()

    s.reset()
    c=s.read_cart()
    assert type(c) == type(None)

In [8]:
test_CartpoleServer()

#### Setup GP

In [ ]:
pset = gp.PrimitiveSet("MAIN", 0)
pset.addPrimitive(ant.if_food_ahead, 2)

pset.addPrimitive(prog2, 2)
pset.addPrimitive(prog3, 3)

pset.addTerminal(get_init_cart_position, 0)
pset.addTerminal(get_init_cart_velocity, 0)
pset.addTerminal(get_init_cart_direction, 0)

pset.addTerminal(ant.turn_left)
pset.addTerminal(ant.turn_right)